In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189) 


# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

In [2]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [3]:
# Build a computational graph.
y = w * x + b    # y = 2 * x + 3
y

tensor(5., grad_fn=<AddBackward0>)

In [4]:
# Compute gradients.
y.backward()

In [5]:
# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

In [60]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)


In [61]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.4824,  0.5668,  0.2724],
        [ 0.5673,  0.0780, -0.0456]], requires_grad=True)
b:  Parameter containing:
tensor([0.1361, 0.4191], requires_grad=True)


In [62]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [63]:
print(linear.parameters())

<generator object Module.parameters at 0x110414048>


In [64]:
# Forward pass.
x.requires_grad_()
pred = linear(x)# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())


loss:  1.2657601833343506


In [10]:
# Backward pass.
loss.backward()

In [11]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)
print ('dL/dx: ', x.grad)


dL/dw:  tensor([[-0.5716, -0.2521,  0.4452],
        [-0.1165,  0.6533,  0.2019]])
dL/db:  tensor([0.8494, 0.1311])
dL/dx:  tensor([[-0.0620,  0.0596, -0.0098],
        [ 0.0380, -0.0426, -0.0215],
        [-0.0518,  0.0446, -0.0319],
        [ 0.0194, -0.0253, -0.0273],
        [-0.1324,  0.1357,  0.0182],
        [-0.0215,  0.0235,  0.0094],
        [ 0.0317, -0.0290,  0.0116],
        [-0.1185,  0.1131, -0.0222],
        [-0.1384,  0.1334, -0.0200],
        [-0.0034, -0.0017, -0.0234]])


In [14]:
# 1-step gradient descent.
optimizer.step()

In [28]:
optimizer_x = torch.optim.SGD([x], lr=0.01)
optimizer_x.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

In [30]:
# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  1.1928918361663818


# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #


In [31]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])


In [32]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

In [33]:
# Convert the torch tensor to a numpy array.
z = y.numpy()

# ================================================================== #
#                         4. Input pipline                           #
# ================================================================== #

In [79]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)
print("train_dataset is ",type(train_dataset)," len is ",len(train_dataset))
print("train_dataset[0] is ",type(train_dataset[0])," len is ",len(train_dataset[0]))
print("train_dataset[0][0] is ",type(train_dataset[0][0]), "shape is ",train_dataset[0][0].shape)
print("train_dataset[0][1] is ",type(train_dataset[0][1])," value is ",train_dataset[0][1])

Files already downloaded and verified
train_dataset is  <class 'torchvision.datasets.cifar.CIFAR10'>  len is  50000
train_dataset[0] is  <class 'tuple'>  len is  2
train_dataset[0][0] is  <class 'torch.Tensor'> shape is  torch.Size([3, 32, 32])
train_dataset[0][1] is  <class 'int'>  value is  6


In [78]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)
print("train_loader is ",type(train_loader)," len is ",len(train_loader))

train_loader is  <class 'torch.utils.data.dataloader.DataLoader'>  len is  782


In [77]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)
print("data_iter is ",type(data_iter)," len is ",len(data_iter))

data_iter is  <class 'torch.utils.data.dataloader._DataLoaderIter'>  len is  782


In [84]:
# Mini-batch images and labels.
images, labels = data_iter.next()
print(type(images),images.shape)
print(type(labels),labels.shape)

<class 'torch.Tensor'> torch.Size([64, 3, 32, 32])
<class 'torch.Tensor'> torch.Size([64])


In [91]:
# Actual usage of the data loader is as below.
i=0
for images, labels in train_loader:
    # Training code should be written here.
    i+=1
print(i)
print(type(images),images.shape) #781*64+16=50000
print(type(labels),labels.shape)

782
<class 'torch.Tensor'> torch.Size([16, 3, 32, 32])
<class 'torch.Tensor'> torch.Size([16])


# ================================================================== #
#                5. Input pipline for custom dataset                 #
# ================================================================== #

In [92]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 


In [ ]:
# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #


In [2]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)
print(type(resnet))

<class 'torchvision.models.resnet.ResNet'>


In [3]:
# If you want to finetune only the top layer of the model, set as below.
i=0

for param in resnet.parameters():
    i+=1
    #print(param.shape)
    param.requires_grad = False
print(i)
for param in resnet.fc.parameters():
    print(param.requires_grad)
    print(param.shape)

62
False
torch.Size([1000, 512])
False
torch.Size([1000])


In [4]:
# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 10)  # 100 is an example.
for param in resnet.fc.parameters():
    print(param.requires_grad)
for param in resnet.layer1.parameters():
    print(param.requires_grad)

True
True
False
False
False
False
False
False
False
False
False
False
False
False


In [14]:
# Forward pass.
images = torch.randn(64, 3, 224, 224)
labels=torch.randn(64,10)
#print (outputs.size())     # (64, 100)

torch.Size([64, 10])


In [18]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.01)

In [22]:
# Forward pass.
outputs = resnet(images)

loss=None
optimizer.zero_grad()
print(resnet.fc.weight.grad)
loss=criterion(outputs,labels)
loss.backward()
optimizer.step()
print(resnet.fc.weight.grad)

tensor([[ 0.0060, -0.0057, -0.0044,  ...,  0.0188,  0.0051, -0.0020],
        [-0.0028, -0.0128,  0.0148,  ..., -0.0039, -0.0158, -0.0052],
        [-0.0093, -0.0125,  0.0048,  ...,  0.0157, -0.0216,  0.0195],
        ...,
        [ 0.0407, -0.0182,  0.0291,  ..., -0.0101,  0.0279, -0.0315],
        [-0.0026,  0.0009, -0.0371,  ...,  0.0188,  0.0104, -0.0125],
        [-0.0163,  0.0028, -0.0290,  ...,  0.0022, -0.0027,  0.0308]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #


In [ ]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')



In [ ]:
# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))